<a href="https://colab.research.google.com/github/tavaresjv/gerador_prova_cientista_de_dados/blob/main/Gerador_de_Provas_de_Cientista_de_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Leitor e Replicador de Provas Semelhantes

# Projeto: Leitor e Replicador de Provas Semelhantes

## Introdução

Este projeto tem como objetivo auxiliar estudantes que estão se preparando para provas, no caso do exemplo desenvolvido, provas de cientista de dados.
O objetivo é usar prints disponíveis online de provas antigas para gerar provas completas utilizando tecnologias avançadas de OCR (Reconhecimento Óptico de Caracteres) e modelos de linguagem natural (LLM).
A partir de imagens de provas antigas, o programa é capaz de tratar as imagens, extrair o texto contido nelas, processá-lo e gerar novas questões de múltipla escolha com conteúdos semelhantes.

### Objetivos Específicos

1. **Leitura de Provas Antigas:** Utilizar o Tesseract OCR para extrair texto de imagens de provas antigas.
2. **Geração de Novas Questões:** Utilizar modelos de linguagem natural, como o LLaMA 3.2-3B, para gerar novas questões de múltipla escolha baseadas no texto extraído.
3. **Criação de Bases de Dados Sintéticas (em andamento):** Gerar novas bases de dados fictícias para questões de aplicação de modelos de machine learning.
4. **Acessibilidade (em andamento):** Tornar o programa acessível através de uma interface amigável, como o Streamlit.

### Tecnologias Utilizadas

- **Google Collab:** Esse código foi escrito e executado orginalmente no Ambiente Google Collab T4 para permitir a realização de processos com alto custo computacional, e portanto ele é customizado para operar nesse ambiente
- **OCR:** Tesseract OCR
- **Processamento de Imagens:** PIL, OpenCV
- **Manipulação de PDFs:** PyPDF2, img2pdf, pymupdf
- **Modelos de Linguagem Natural:** Transformers, LLaMA 2
- **Criação de PDFs:** FPDF

### Estrutura do Projeto

1. **Pré-processamento de Imagens:** Tratamento das imagens para melhorar a qualidade do OCR.
2. **Extração de Texto:** Extração do texto das imagens processadas.
3. **Geração de Questões:** Utilização de modelos de linguagem natural para gerar novas questões.
4. **Armazenamento e Acesso:** Salvamento dos resultados no Google Drive para facilitar o acesso futuro.

Este projeto visa não apenas facilitar o estudo de provas antigas, mas também proporcionar uma maneira inovadora de gerar novas questões e bases de dados, contribuindo para um aprendizado mais eficaz e personalizado.

In [ ]:
# Instala todas dependencias que serão usadas no projeto

!sudo apt install tesseract-ocr
!sudo apt install libtesseract-dev
!sudo apt install tesseract-ocr-por
!pip install numpy pillow img2pdf pymupdf pytesseract transformers torch accelerate

In [ ]:
# Configura o drive onde serão salvos alguns documentos, para facilitar caso seja necessário rodar o código mais de uma vez
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Objetivo:** Esse projeto visa auxiliar os estudantes de provas de cientista de dados a partir de prints de uma prova antiga.

-  Iremos ler uma prova antiga do processo seletivo de cientista de dados do banco Itau através do Tesseract OCR. Esse texto, juntamente com um prompt engenheirado serão o alimento para que nosso algoritmo seja capaz de criar uma prova de cientista de dados diferente, mas com conteúdos semelhantes.

- Um feature extra é que o programa também seja capaz de gerar novas bases de dados fictícias para as questões de aplicação de modelos de machine learning. Idealmente o programa final será acessível por Streamlit

# 0.Bibliotecas e configurações iniciais

In [ ]:
# Analise e tratamento de dados
import numpy as np

# Interação com arquivos e diretórios

import os

# Processamento de Imagens e PDF's
from PIL import Image, ImageFilter  # Para processamento de imagens
import img2pdf  # Converte imagens em PDFs
import fitz  # Mnipula PDFs
import pytesseract  # Realiza OCR em imagens
import io  # Para manipulação de bytes e fluxo de dados

# LLMs
from transformers import pipeline
import torch
import accelerate


# Criacao de bases de dados sintéticas
# Em andamento

In [ ]:
# Configuramos o path do Tesseract-OCR em nossa maquina, ja que ele nao esta instalado na root
#pytesseract.pytesseract.tesseract_cmd = r"C:\Users\josev\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

os.environ['TESSDATA_PREFIX'] = '/usr/share/tesseract-ocr/4.00/tessdata'
print(os.getenv('TESSDATA_PREFIX'))

/usr/share/tesseract-ocr/4.00/tessdata


# 1. Processamento de imagens PDF's

Uma premissa que devemos levar em consideracao e que o OCR pode nao ser perfeito, e que o texto pode estar com uma qualidade ruim, o que torna a leitura do OCR imperfeita

Para evitar erros e perda de informacao, mas manter o pdf original intocado para que possa ser lido por um humano, devemos realizar alguns tratamentos no pdf

-  Como parte do pre-processamento das imagens, para facilitar a leitura do texto, iremos realizar os seguintes tratamentos:
    - **Escala de Cinza:** dessa forma reduzimos o ruido de formatacao e fontes e melhoramos o contraste do texto
    - **Binarizar:** dessa forma as imagens ficaram em preto e branco, o que melhora a visibilidade do texto
    - **Ajustar tamanho:** uma vez que cada print pode estar com um tamanho distinto
    - **Ruidos:** iremos tratar os ruidos da imagem usando o metodo .filter e melhorando a nitidez das fotos

In [ ]:
# Cria uma funcao que aplica as tecnicas e preprocessamento citadas

def preprocess_image(image_path):
    with Image.open(image_path) as img:
        img = img.convert('L') # Converte a imagem para escala de cinza
        img = img.resize((img.width*2,img.height*2), Image.Resampling.LANCZOS) # Aumenta aresolucao da  imagem
        img = img.filter(ImageFilter.SHARPEN) # Aplica um filtro de nitidez
        return img

# Cria uma funcao que transforma as imagens em uma pasta em um PDF
def image_to_pdf(image_path,output_pdf_path):

    images = []
    for img_name in os.listdir(image_path):
        if img_name.lower().endswith(('.png','.jpg','.jpeg')):
            img_path = os.path.join(image_path,img_name)
            preprocessed_image_path = os.path.join(image_path,'preprocessed',img_name)
            if not os.path.exists(os.path.join(image_path,'preprocessed')):
                os.makedirs(os.path.join(image_path,'preprocessed'))
            img = preprocess_image(img_path)
            img.save(preprocessed_image_path)
            images.append(preprocessed_image_path)

    images.sort()
    with open(output_pdf_path,'wb') as f:
        f.write(img2pdf.convert(images));
    print(f"PDF criado com sucesso em {output_pdf_path}")

# Exemplo
pasta_imagens = r"/content"
caminho_saida = r"/content/pre_processed"

image_to_pdf(pasta_imagens, caminho_saida)

PDF criado com sucesso em /content/pre_processed


Agora que temos as imagens tratadas e concatenadas em um pdf, podemos passar elas pelo OCR

In [ ]:
# Definiremos uma função que extrai o texto de um PDF sem senha
def extract_text_from_pdf(pdf_file):
    complete_text = ''
    document = fitz.open(pdf_file)

    # Iteramos sobre as paginas do df com as funcoes load_page e get_images, para extrair textos e imagens de cada pagina
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        images = page.get_images(full=True)

        # Agora iteramos sobre as imagens identificadas na pagina
        for img_index, img in enumerate(images):
            xref = img[0] # Essa referencia indica qual imagem esta sendo extraida
            image = document.extract_image(xref)
            image_bytes = image["image"]

            # Agora que convertemos as imagens para um formato que o Tesseract compreenda
            with Image.open(io.BytesIO(image_bytes)) as img_pil:
                text = pytesseract.image_to_string(img_pil, lang = 'por') # E extraimos o texto das imagens convertidas
                complete_text += text + '\n'

    print ('Texto extraido com sucesso')
    print (complete_text)
    return complete_text

# Exemplo:
pdf_path = caminho_saida
extracted = extract_text_from_pdf(caminho_pdf)
extracted

Texto extraido com sucesso
PERGUNTA 1 DE 37
(0.2 ponto) Uma cidade com 200 mil habitantes foi dividida em 30 bairros distintos. Por meio de um sorteio, foram
selecionados 15 bairros e todos os moradores desses bairros foram entrevistados. Esse tipo de delineamento é
característico de uma amostragem:

 

ecione torção O

O Alestória smpies
Estratificada
Por conglomerado
sistemática

Não quero responder

af

 

PERGUNTA 2 DE 37
(0,2 ponto) A probabilidade de um ponto escolhido ao acaso no segmento de reta [0,2 ] estar entre 1 e 3/2 é igual a:

SELECIONE 1 OPÇÃO O

Não quero responder

PERGUNTA 3 DE37

(0,2 ponto) Em uma caixa existem 6 bolas brancas e 4 bolas pretas. Suponha que 2 bolas são selecionadas em
sequência e sem reposição dessa caixa. A probabilidade de selecionarmos uma bola branca e uma preta é:

SELECIONE 1 OPÇÃO O

9/5

B/14

9/16

Q ss

Não quero responder

PERGUNTA 4 DE 37

(0,2 ponto) Sejam A e B duas distribuições de probabilidade assimétricas. A distribuição A apres

'PERGUNTA 1 DE 37\n(0.2 ponto) Uma cidade com 200 mil habitantes foi dividida em 30 bairros distintos. Por meio de um sorteio, foram\nselecionados 15 bairros e todos os moradores desses bairros foram entrevistados. Esse tipo de delineamento é\ncaracterístico de uma amostragem:\n\n \n\necione torção O\n\nO Alestória smpies\nEstratificada\nPor conglomerado\nsistemática\n\nNão quero responder\n\naf\n\n \n\x0c\nPERGUNTA 2 DE 37\n(0,2 ponto) A probabilidade de um ponto escolhido ao acaso no segmento de reta [0,2 ] estar entre 1 e 3/2 é igual a:\n\nSELECIONE 1 OPÇÃO O\n\nNão quero responder\n\x0c\nPERGUNTA 3 DE37\n\n(0,2 ponto) Em uma caixa existem 6 bolas brancas e 4 bolas pretas. Suponha que 2 bolas são selecionadas em\nsequência e sem reposição dessa caixa. A probabilidade de selecionarmos uma bola branca e uma preta é:\n\nSELECIONE 1 OPÇÃO O\n\n9/5\n\nB/14\n\n9/16\n\nQ ss\n\nNão quero responder\n\x0c\nPERGUNTA 4 DE 37\n\n(0,2 ponto) Sejam A e B duas distribuições de probabilidade assimét

In [ ]:
# Agora iremos salvar as imagens, pdf e texto extraido no Google Drive para facilitar futuros acessos

text_path = '/content/drive/My Drive/Collab dados/extracted.txt'
with open(text_path, 'w') as file:
  file.write(extracted)

# 2. Integração com o LLM (LLama 2 70B)

### Como carregar e usar o LLaMA 2:

O LLaMA 2 pode ser acessado via **Hugging Face Transformers**

Você **precisará de uma GPU** para rodar o modelo de forma eficiente (se não tiver, pode **usar um modelo menor, como o LLaMA 7B**, ou até mesmo a API da Hugging Face).

### Prompt Engineering:

Como criar **prompts eficazes para o LLM gerar questões**

- **Exemplo de prompt:** "Com base no texto a seguir, gere 5 questões de múltipla escolha sobre machine learning."

### Integração com o código existente:

Como passar o texto extraído para o LLM e receber as questões geradas.

In [ ]:
# Prmieiro iremos carregar o arquivo importando-o do Drive

drive.mount('/content/drive')

text_path = '/content/drive/My Drive/Collab dados/extracted.txt'

with open(text_path, 'r') as file:
  extracted = file.read()

  print(extracted)

In [ ]:
# Agora carregamos o tokenizer e o modelo

model_name = "meta-llama/Llama-3.2-3B-Instruct"
model_token = "Seu_token_huggingface"

tokenizer = AutoTokenizer.from_pretrained(model_name, token=model_token)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=model_token,
    torch_dtype=torch.float16,
    device_map="auto"  # Usar GPU se disponível, caso contrário, CPU
)

# Configurar o pad_token como o eos_token
tokenizer.pad_token = tokenizer.eos_token

# Definimos uma função que gera questões, com base no texto extraído da prova antiga
def generate_questions(text, num_questions):
  try:
    prompt = f"""
            Você é um professor de ciência de dados. Sua tarefa é gerar {num_questions} questões de múltipla escolha sobre ciência de dados, com base no texto fornecido. Siga as regras abaixo:

            1. Cada questão deve ter 4 opções de resposta.
            2. Apenas uma opção deve ser correta.
            3. Indique a resposta correta com um * ao lado da opção.
            4. As questões devem ser claras e diretamente relacionadas ao texto.
            5. Use o texto abaixo como base para criar as questões.

            Texto:
            {text}

            Exemplo de formato:
            1. Qual é o objetivo principal da regressão linear?
            a) Classificar dados
            b) Prever valores contínuos *
            c) Agrupar dados
            d) Reduzir dimensionalidade

            Questões:
            """

    # Tokenizar o prompt com truncamento e padding
    inputs = tokenizer(
        prompt,
        return_tensors='pt',
        truncation=True,  # Truncar o texto para o limite de tokens do modelo
        max_length=1024,  # Define o limite de tokens do modelo (1024 tokens)
        padding='max_length'  # Preenche o texto com padding para que todos os textos tenham o mesmo comprimento
    ).to(model.device)  # Mover os inputs para o mesmo dispositivo do modelo
    print(f"inputs definidos \n")

    # Gerar as questões
    outputs = model.generate(
        inputs['input_ids'],
        max_new_tokens = 500,  # Ajuste o comprimento máximo conforme necessário
        num_return_sequences=num_questions,
        temperature=0.3,  # Controle a aleatoriedade das saídas
        top_p=0.9,  # Controla a diversidade de respostas
        do_sample=True,  # Ativa a amostragem de tokens
        pad_token_id=tokenizer.pad_token_id,  # Usar o pad_token_id configurado
        attention_mask=inputs['attention_mask'],  # Passar a máscara de atenção
        repetition_penalty = 1.2 # Assim evitamos repeticoes repetitivas, comuns quando o modeloa alucina
    )
    print(f"outputs definidos \n")

    # Decodificar a resposta
    questions = [tokenizer.decode(output, skip_special_tokens = True) for output in outputs]
    return questions
  except Excepetion as e:
    print(f"Erro ao gerar as questões: {e}")
    return []


# Exemplo:
test_text = extracted
questions = generate_questions(test_text, num_questions=1)

print('Questões geradas:')
for i, question in enumerate(questions, 1):
  print(f"Questão {i}:\n {question}\n")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

inputs definidos 

outputs definidos 

Questões geradas:
Questão 1:
 
            Você é um professor de ciência de dados. Sua tarefa é gerar 1 questões de múltipla escolha sobre ciência de dados, com base no texto fornecido. Siga as regras abaixo:

            1. Cada questão deve ter 4 opções de resposta.
            2. Apenas uma opção deve ser correta.
            3. Indique a resposta correta com um * ao lado da opção.
            4. As questões devem ser claras e diretamente relacionadas ao texto.
            5. Use o texto abaixo como base para criar as questões.

            Texto:
            PERGUNTA 1 DE 37
(0.2 ponto) Uma cidade com 200 mil habitantes foi dividida em 30 bairros distintos. Por meio de um sorteio, foram
selecionados 15 bairros e todos os moradores desses bairros foram entrevistados. Esse tipo de delineamento é
característico de uma amostragem:

 

ecione torção O

O Alestória smpies
Estratificada
Por conglomerado
sistemática

Não quero responder

af

 

PERG

In [ ]:
model_name = "meta-llama/Llama-3.2-3B-Instruct"
model_token = "Seu_token_huggingface"

pipe = pipeline(
    "text-generation",
    model= model_name,
    torch_dtype = torch.bfloat16,
    device_map = "cuda",
    )

messages = [
    {"role":"system", "content": """
    Você é um professor de ciência de dados. Sua tarefa é gerar uma questão de múltipla escolha sobre ciência de dados, com base no assunto fornecido.
    Siga as regras abaixo:
    1. Cada questão deve ter 4 opções de resposta.
    2. Apenas uma opção deve ser correta.
    3. Indique a resposta correta com um * ao lado da opção.
    4. As questões devem ser claras e diretamente relacionadas ao texto.
    5. Use o texto abaixo como base para criar as questões.
    6. Não dê alternativas iguais
    """}, # O contexto que é passado para a geracao de texto


    {"role":"system", "content": "Me dê uma questão complicada sobre regressão linear"}, # A tarefa designada
    ]

outputs = pipe(
    messages,
    max_new_tokens = 500,
    )
question_final_output = outputs[0]["generated_text"][2]['content']

print(question_final_output)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Aqui está uma questão de múltipla escolha sobre regressão linear:

**Questão**

Um cientista de dados está analisando os dados de vendas de um produto em diferentes regiões. Ele descobriu que as vendas estão diretamente relacionadas à população da região e à temperatura média anual. Ele deseja criar um modelo de regressão linear para prever as vendas de um novo produto em uma região desconhecida.

O cientista de dados decide usar as seguintes variáveis:

* Y: vendas do produto
* X1: população da região
* X2: temperatura média anual

Quais são os principais problemas que o cientista de dados deve considerar ao criar o modelo de regressão linear?

**Opções**

A) O cientista de dados deve usar um modelo de regressão linear simples, sem considerar a interação entre as variáveis.
B) O cientista de dados deve usar um modelo de regressão linear múltipla, considerando apenas a relação entre as vendas e a população da região.
C) O cientista de dados deve usar um modelo de regressão linear múlti

In [ ]:
model_name = "tiiuae/falcon-rw-1b"
model_token = "Seu_token_huggingface"

pipe = pipeline(
    "text-generation",
    model= model_name,
    torch_dtype = torch.bfloat16,
    device_map = "cuda",
    )

messages = [
    {"role":"system", "content": """
    Você é um professor de ciência de dados. Sua tarefa é gerar uma questão de múltipla escolha sobre ciência de dados, com base no assunto fornecido.
    Siga as regras abaixo:
    1. Cada questão deve ter 4 opções de resposta.
    2. Apenas uma opção deve ser correta.
    3. Indique a resposta correta com um * ao lado da opção.
    4. As questões devem ser claras e diretamente relacionadas ao texto.
    5. Use o texto abaixo como base para criar as questões.
    6. Não dê alternativas iguais
    """}, # O contexto que é passado para a geracao de texto


    {"role":"system", "content": "Me dê uma questão complicada sobre regressão linear"}, # A tarefa designada
    ]

outputs = pipe(
    messages,
    max_new_tokens = 500,
    )
question_final_output = outputs[0]["generated_text"][2]['content']

print(question_final_output)

In [ ]:
final_question = outputs[0]["generated_text"][2]['content']
print(final_question)

Aqui está uma questão de múltipla escolha sobre regressão linear:

**Questão**

Um modelo de regressão linear é usado para prever o valor de vendas de um produto com base no preço e na quantidade disponível. Os dados de treinamento são os seguintes:

| Preço | Quantidade | Vendas |
| --- | --- | --- |
| 10 | 100 | 1000 |
| 20 | 200 | 2000 |
| 30 | 300 | 3000 |
| 40 | 400 | 4000 |

Quem é o parâmetro mais importante para o modelo de regressão linear nesse cenário?

**Opções**

A) R-squared (*)
B) Coeficiente de determinação (R²)
C) Coeficiente de regressão (β)
D) Erro médio absoluto (MAE)

Escolha a opção correta!


In [ ]:
test_text = """"Aqui está uma questão de múltipla escolha sobre regressão linear:

**Questão**

Um cientista de dados está analisando os dados de vendas de um produto em diferentes regiões. Ele descobriu que as vendas estão diretamente relacionadas à população da região e à temperatura média anual. Ele deseja criar um modelo de regressão linear para prever as vendas de um novo produto em uma região desconhecida.

O cientista de dados decide usar as seguintes variáveis:

* Y: vendas do produto
* X1: população da região
* X2: temperatura média anual

Quais são os principais problemas que o cientista de dados deve considerar ao criar o modelo de regressão linear?

**Opções**

A) O cientista de dados deve usar um modelo de regressão linear simples, sem considerar a interação entre as variáveis.
B) O cientista de dados deve usar um modelo de regressão linear múltipla, considerando apenas a relação entre as vendas e a população da região.
C) O cientista de dados deve usar um modelo de regressão linear múltipla, considerando a relação entre as vendas e a temperatura média anual, bem como a interação entre as variáveis.
D) O cientista de dados deve usar um modelo de regressão linear múltipla, considerando a relação entre as vendas e a temperatura média anual, bem como a interação entre as variáveis."""
# Tokenizar o texto e contar os tokens
tokens = tokenizer.tokenize(test_text)
num_tokens = len(tokens)

num_tokens

328

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Salvar o modelo e o tokenizer no Google Drive
model.save_pretrained("/content/drive/MyDrive/falcon_model")
tokenizer.save_pretrained("/content/drive/MyDrive/falcon_tokenizer")